# Protocol PDF 解析器 - 示範筆記本

此筆記本展示如何使用 Protocol Parser 模組從臨床試驗 Protocol PDF 中提取關鍵資訊。

## 功能特點
- ✅ 自動提取 Protocol 關鍵資訊
- ✅ 使用 Google Gemini AI 進行智能解析
- ✅ 輸出結構化 JSON 格式
- ✅ 支持免費 API

---

## 1. 安裝依賴套件

In [ ]:
# 安裝必要的套件
!pip install -q pdfplumber google-generativeai python-dotenv

## 2. 導入模組

In [ ]:
import sys
import os
from pathlib import Path
import json

# 添加模組路徑（如果在 Colab 中運行）
if 'google.colab' in sys.modules:
    # 在 Colab 中需要先克隆或上傳專案
    # 這裡假設已經上傳了模組檔案
    pass

# 導入 Protocol Parser
try:
    from modules.protocol_parser import ProtocolParser, ProtocolInfo
    print("✓ 模組導入成功")
except ImportError as e:
    print(f"✗ 模組導入失敗: {e}")
    print("請確保 modules/protocol_parser.py 檔案存在")

## 3. 設置 API 金鑰

### 獲取免費的 Gemini API 金鑰:
1. 訪問 https://makersuite.google.com/app/apikey
2. 使用 Google 帳號登入
3. 點擊 "Create API Key"
4. 複製 API 金鑰並貼到下方

In [ ]:
# 方式 1: 直接設置（僅供測試）
GEMINI_API_KEY = "YOUR_API_KEY_HERE"  # 請替換為您的實際 API 金鑰

# 方式 2: 使用 Colab 的 userdata（推薦）
# from google.colab import userdata
# GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# 設置環境變數
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

if GEMINI_API_KEY and GEMINI_API_KEY != "YOUR_API_KEY_HERE":
    print("✓ API 金鑰已設置")
else:
    print("⚠ 請設置有效的 API 金鑰")

## 4. 上傳 PDF 檔案

請上傳您要解析的 Protocol PDF 檔案

In [ ]:
# 如果在 Colab 中，可以使用檔案上傳
if 'google.colab' in sys.modules:
    from google.colab import files
    print("請選擇 Protocol PDF 檔案:")
    uploaded = files.upload()
    
    if uploaded:
        pdf_filename = list(uploaded.keys())[0]
        pdf_path = pdf_filename
        print(f"✓ 已上傳: {pdf_filename}")
    else:
        print("未選擇檔案")
        pdf_path = None
else:
    # 本地環境，請指定檔案路徑
    pdf_path = "path/to/your/protocol.pdf"  # 請替換為實際路徑
    print(f"使用本地檔案: {pdf_path}")

## 5. 初始化解析器

In [ ]:
# 初始化 Protocol Parser
parser = ProtocolParser(
    api_key=GEMINI_API_KEY,
    model_name="gemini-1.5-flash"  # 使用免費的 Flash 模型
)

print("✓ Protocol Parser 已初始化")

## 6. 解析 Protocol PDF

這個步驟會:
1. 讀取 PDF 文本
2. 使用 AI 提取關鍵資訊
3. 結構化輸出結果

處理時間取決於 PDF 大小，通常需要 30-60 秒

In [ ]:
if pdf_path and Path(pdf_path).exists():
    try:
        print("開始解析 Protocol...")
        print("(這可能需要 30-60 秒，請稍候)\n")
        
        # 解析 Protocol
        # 選項: max_pages=50 只讀取前 50 頁以加快速度
        protocol_info = parser.parse_protocol(
            pdf_path=pdf_path,
            max_pages=None  # 讀取全部頁面
        )
        
        print("\n✓ Protocol 解析完成!")
        
    except Exception as e:
        print(f"✗ 解析失敗: {e}")
        import traceback
        traceback.print_exc()
        protocol_info = None
else:
    print("✗ 請先上傳 PDF 檔案")
    protocol_info = None

## 7. 顯示提取結果

### 7.1 基本資訊

In [ ]:
if protocol_info:
    print("=" * 80)
    print("基本資訊")
    print("=" * 80)
    print(f"\n試驗標題: {protocol_info.study_title or 'N/A'}")
    print(f"Protocol 編號: {protocol_info.protocol_number or 'N/A'}")
    print(f"贊助商: {protocol_info.sponsor or 'N/A'}")
    print(f"試驗階段: {protocol_info.phase or 'N/A'}")
    print(f"試驗設計: {protocol_info.study_design or 'N/A'}")
    print(f"目標族群: {protocol_info.target_population or 'N/A'}")
    print(f"樣本數: {protocol_info.sample_size or 'N/A'}")

### 7.2 訪視時程

In [ ]:
if protocol_info and protocol_info.visit_schedule:
    print("\n=" * 80)
    print("訪視時程")
    print("=" * 80)
    for i, visit in enumerate(protocol_info.visit_schedule, 1):
        print(f"{i:2d}. {visit}")
else:
    print("未提取到訪視時程")

### 7.3 終點指標

In [ ]:
if protocol_info:
    print("\n=" * 80)
    print("主要終點指標")
    print("=" * 80)
    if protocol_info.primary_endpoints:
        for i, endpoint in enumerate(protocol_info.primary_endpoints, 1):
            print(f"{i}. {endpoint}")
    else:
        print("未提取到主要終點指標")
    
    print("\n=" * 80)
    print("次要終點指標")
    print("=" * 80)
    if protocol_info.secondary_endpoints:
        for i, endpoint in enumerate(protocol_info.secondary_endpoints, 1):
            print(f"{i}. {endpoint}")
    else:
        print("未提取到次要終點指標")

### 7.4 納入/排除標準

In [ ]:
if protocol_info:
    print("\n=" * 80)
    print("納入標準")
    print("=" * 80)
    if protocol_info.inclusion_criteria:
        for i, criterion in enumerate(protocol_info.inclusion_criteria, 1):
            print(f"{i:2d}. {criterion}")
    else:
        print("未提取到納入標準")
    
    print("\n=" * 80)
    print("排除標準")
    print("=" * 80)
    if protocol_info.exclusion_criteria:
        for i, criterion in enumerate(protocol_info.exclusion_criteria, 1):
            print(f"{i:2d}. {criterion}")
    else:
        print("未提取到排除標準")

### 7.5 CRF 領域

In [ ]:
if protocol_info and protocol_info.crf_domains:
    print("\n=" * 80)
    print("所需 CRF 領域")
    print("=" * 80)
    
    # 以表格形式顯示
    domains = protocol_info.crf_domains
    cols = 3
    for i in range(0, len(domains), cols):
        row = domains[i:i+cols]
        print("  ".join(f"{d:30}" for d in row))
else:
    print("未提取到 CRF 領域")

## 8. 完整 JSON 輸出

In [ ]:
if protocol_info:
    print("\n=" * 80)
    print("完整 JSON 輸出")
    print("=" * 80)
    print(protocol_info.to_json())

## 9. 保存結果

In [ ]:
if protocol_info:
    # 創建輸出目錄
    output_dir = Path("output")
    output_dir.mkdir(exist_ok=True)
    
    # 生成檔案名稱
    if protocol_info.protocol_number:
        output_filename = f"{protocol_info.protocol_number}_info.json"
    else:
        output_filename = "protocol_info.json"
    
    output_path = output_dir / output_filename
    
    # 保存 JSON
    parser.save_to_json(protocol_info, str(output_path))
    print(f"✓ 結果已保存至: {output_path}")
    print(f"  檔案大小: {output_path.stat().st_size:,} bytes")
    
    # 在 Colab 中可以下載檔案
    if 'google.colab' in sys.modules:
        from google.colab import files
        print("\n下載 JSON 檔案:")
        files.download(str(output_path))

## 10. 資料品質檢查

In [ ]:
if protocol_info:
    print("\n=" * 80)
    print("資料品質檢查")
    print("=" * 80)
    
    fields = {
        "試驗標題": protocol_info.study_title,
        "Protocol 編號": protocol_info.protocol_number,
        "贊助商": protocol_info.sponsor,
        "試驗階段": protocol_info.phase,
        "試驗設計": protocol_info.study_design,
        "目標族群": protocol_info.target_population,
        "樣本數": protocol_info.sample_size,
        "訪視時程": protocol_info.visit_schedule,
        "主要終點": protocol_info.primary_endpoints,
        "次要終點": protocol_info.secondary_endpoints,
        "納入標準": protocol_info.inclusion_criteria,
        "排除標準": protocol_info.exclusion_criteria,
        "CRF 領域": protocol_info.crf_domains,
    }
    
    extracted_count = sum(1 for v in fields.values() if v)
    total_count = len(fields)
    completion_rate = (extracted_count / total_count) * 100
    
    print(f"\n提取完整度: {extracted_count}/{total_count} ({completion_rate:.1f}%)\n")
    
    print("欄位狀態:")
    for field_name, field_value in fields.items():
        status = "✓" if field_value else "✗"
        value_info = ""
        if isinstance(field_value, list):
            value_info = f"({len(field_value)} 項)"
        elif isinstance(field_value, str):
            value_info = f"({len(field_value)} 字元)"
        
        print(f"  {status} {field_name:15} {value_info}")

## 總結

✅ 本筆記本展示了如何使用 Protocol Parser 模組提取 Protocol 關鍵資訊  
✅ 結果已保存為 JSON 格式，可用於後續處理  
✅ 提取的資訊包括試驗設計、終點指標、納排標準和 CRF 需求  

### 下一步
- 使用提取的資訊生成 CRF 模板
- 建立試驗資料庫結構
- 自動化文件生成流程

---

**需要幫助？** 查看 `README_PROTOCOL_PARSER.md` 獲取完整文檔